```
esalina@2018comp:/mnt/five_tera/PlantsApp/eddie_docker$ docker build .
Sending build context to Docker daemon  2.048kB
Step 1/1 : FROM jupyter/tensorflow-notebook:4a112c0f11eb
 ---> 955c541a5ce1
Successfully built 955c541a5ce1
esalina@2018comp:/mnt/five_tera/PlantsApp/eddie_docker$ cat Dockerfile 
FROM jupyter/tensorflow-notebook:4a112c0f11eb
esalina@2018comp:/mnt/five_tera/PlantsApp/eddie_docker$ cd ..
esalina@2018comp:/mnt/five_tera/PlantsApp$ history |grep -Pi '955c541a5ce1'
 1901  docker run -p 8888:8888 -v ${PWD}:/home/jovyan/work   -it 955c541a5ce1 /bin/bash
 2012  history |grep -Pi '955c541a5ce1'
esalina@2018comp:/mnt/five_tera/PlantsApp$ docker run -p 8888:8888 -v ${PWD}:/home/jovyan/work   -it 955c541a5ce1 /bin/bash
```


In [1]:
# %tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import applications,optimizers, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
#from keras_applications.imagenet_utils import _obtain_input_shape
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, Callback

In [2]:
def model_dropout_before_flatten():
    '''
    A LeNet inspired CNN that gradually decreases in image size, but increases 
    in channel depth. We end with Dense layers to bring flattened representation gradually down
    to 30, which is the number of target outputs
    
    Model uses ReLu activation functions, and the Adam optimizer
    '''
    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation="relu", input_shape = (96,96,1)))
    model.add(Conv2D(16, (3, 3), activation="relu", input_shape = (96,96,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation="relu"))
    model.add(Conv2D(32, (3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(30, activation=None))
    model.compile(optimizer='adam',
                  loss='mean_squared_error')
    return model

In [3]:
weights_file='final.h5'
model_iterative = model_dropout_before_flatten()
model_iterative.load_weights(weights_file)

In [4]:
#https://stackoverflow.com/questions/47266383/save-and-load-weights-in-keras

In [5]:
filepath=weights_file+".wam.dat"
model_iterative.save(filepath)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: final.h5.wam.dat/assets


In [6]:
#import tensorflow as tf
#converter = tf.lite.TFLiteConverter.from_keras_model_file('crop.h5') 
converter=tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(filepath)
tfmodel = converter.convert() 
open ("output.tflite" , "wb") .write(tfmodel)

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
INFO:tensorflow:Assets written to: /tmp/tmp4q03h3e1/assets
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /tmp/tmp4q03h3e1/variables/variables
INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default', '__saved_model_init_op'}
INFO:tensorflow:input tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: conv2d_input
INFO:tensorflow: tensor name: serving_default_conv2d_input:0, shape: (-1, 96, 96, 1), type: DT_FLOAT
INFO:tensorflow:output tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: dense_3
INFO:tensorflow: tensor

9355968